## Set up

### imports

In [1]:

import sys
import numpy as np
import keras
from sklearn.metrics import classification_report, accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
import pandas as pd
import gc
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.append('../src')
from data.utils import load
from models.create_models import create_cnn_model, create_fcnn_model
from keras import backend as K


keras.utils.set_random_seed(42)

2024-09-01 12:25:37.436129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 12:25:37.490514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 12:25:37.504605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 12:25:37.570905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-01 12:25:38.797591: W tensorflow/compiler/tf2

### definitions

In [2]:
data_dir = "../data/"

# input image dimensions
img_rows, img_cols = 28, 28


PATH_MODELS = "../models/"
PATH_LOGS = "../logs/"
RESULTS_PATH = "../results/"

BATCH_SIZE = 128
EPOCHS = 1000

### Carregar dataset

In [3]:
# Load the data
X_train = load(data_dir + 'kmnist-train-imgs.npz')
X_test = load(data_dir + 'kmnist-test-imgs.npz')
y_train = load(data_dir + 'kmnist-train-labels.npz')
y_test = load(data_dir + 'kmnist-test-labels.npz')


if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('{} train samples, {} test samples'.format(len(X_train), len(X_test)))

# coleta dos valores unicos e das contagens
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

num_classes = len(unique_train) if list(unique_train) == list(unique_test) else None



# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples, 10000 test samples


### Criar conjunto aumentado

In [4]:
# Crie o gerador de dados com data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Supomos que você tenha suas imagens de treino em x_train e labels em y_train
datagen.fit(X_train)

# Defina o número de amostras aumentadas que você deseja criar
num_augmented_samples = len(X_train) * 10  # por exemplo, 10 vezes o conjunto original

# Inicialize arrays para armazenar os dados aumentados
X_train_augmented = np.zeros((num_augmented_samples, *X_train.shape[1:]), dtype=np.float32)
y_train_augmented = np.zeros((num_augmented_samples, *y_train.shape[1:]), dtype=np.float32)

# Gere os dados aumentados
i = 0
for x_batch, y_batch in datagen.flow(X_train, y_train, batch_size=BATCH_SIZE):
    if i >= num_augmented_samples:
        break
    batch_size_aux = x_batch.shape[0]
    X_train_augmented[i:i+batch_size_aux] = x_batch
    y_train_augmented[i:i+batch_size_aux] = y_batch
    i += batch_size_aux

## Analysis

In [5]:
print("Num class train: ", len(unique_train))
print("Num class test: ", len(unique_test))


print('### Treino ###')
for i in range(len(unique_train)): print(f'classe {unique_train[i]}: {counts_train[i]} observações')
print('\n### Teste ###')
for i in range(len(unique_test)): print(f'classe {unique_test[i]}: {counts_test[i]} observações')

Num class train:  10
Num class test:  10
### Treino ###
classe 0: 6000 observações
classe 1: 6000 observações
classe 2: 6000 observações
classe 3: 6000 observações
classe 4: 6000 observações
classe 5: 6000 observações
classe 6: 6000 observações
classe 7: 6000 observações
classe 8: 6000 observações
classe 9: 6000 observações

### Teste ###
classe 0: 1000 observações
classe 1: 1000 observações
classe 2: 1000 observações
classe 3: 1000 observações
classe 4: 1000 observações
classe 5: 1000 observações
classe 6: 1000 observações
classe 7: 1000 observações
classe 8: 1000 observações
classe 9: 1000 observações


## Classification

In [6]:
METRICS =[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.FalseNegatives(name='fn')
    ]

I0000 00:00:1725204470.237967 1136349 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725204470.270369 1136349 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725204470.273415 1136349 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725204470.277126 1136349 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

### CNN Benchmark

#### Normal

In [7]:
# callbacks do modelo benchmark
callbacks_benchmark = [
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_benchmark.keras', save_best_only=True, monitor='val_loss'),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_benchmark.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_benchmark = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics= METRICS
)

# treina modelo
history_benchmark = model_benchmark.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks = callbacks_benchmark
          )

# realiza predicao
train_pred_proba = model_benchmark.predict(X_train)
test_pred_proba = model_benchmark.predict(X_test)

# salva predicao no treino
test_results_benchmark = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_benchmark.to_csv(RESULTS_PATH + 'test/cnn_benchmark_test_results.csv', index = False)

# salva predicao no teste
train_results_benchmark = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_benchmark.to_csv(RESULTS_PATH + 'train/cnn_benchmark_train_results.csv', index = False)

json.dump(history_benchmark.history, open(RESULTS_PATH + "history/benchmark_cnn_history.json", "w"))


/home/semcovici/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-01 12:27:51.273002: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-09-01 12:27:51.381000: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/12


I0000 00:00:1725204473.804051 1137925 service.cc:146] XLA service 0x749c8800e170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725204473.804106 1137925 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-09-01 12:27:53.868722: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-01 12:27:54.030289: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  1/469 ━━━━━━━━━━━━━━━━━━━━ 37:36 5s/step - accuracy: 0.1250 - auc: 0.5144 - fn: 128.0000 - fp: 0.0000e+00 - loss: 2.3012 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 1152.0000 - tp: 0.0000e+00

I0000 00:00:1725204476.441744 1137925 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.1233 - auc: 0.5476 - fn: 30143.5918 - fp: 0.0000e+00 - loss: 2.2922 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 271292.3125 - tp: 0.0000e+00 - val_accuracy: 0.1686 - val_auc: 0.6450 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.2700 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00
Epoch 2/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1778 - auc: 0.6462 - fn: 30143.5918 - fp: 0.0000e+00 - loss: 2.2544 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 271292.3125 - tp: 0.0000e+00 - val_accuracy: 0.2549 - val_auc: 0.7155 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.2363 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00
Epoch 3/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2548 - auc: 0.7184 - fn: 30143.5918 - fp: 0.0000e+00 - loss: 2.2071 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 27

2024-09-01 12:28:47.186304: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-09-01 12:28:47.298919: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 745us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


#### Com Data Augmentation

In [8]:
# callbacks do modelo benchmark
callbacks_benchmark_aug = [
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_aug_benchmark.keras', save_best_only=True, monitor='val_loss'),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_aug_benchmark.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_benchmark_aug = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics= METRICS
)

# treina modelo
history_benchmark_aug = model_benchmark_aug.fit(X_train_augmented, y_train_augmented,
          batch_size=BATCH_SIZE,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks = callbacks_benchmark_aug
          )

# realiza predicao
train_pred_proba = model_benchmark_aug.predict(X_train)
test_pred_proba = model_benchmark_aug.predict(X_test)

# salva predicao no treino
test_results_benchmark_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_benchmark_aug.to_csv(RESULTS_PATH + 'test/cnn_benchmark_aug_test_results.csv', index = False)

# salva predicao no teste
train_results_benchmark_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_benchmark_aug.to_csv(RESULTS_PATH + 'train/cnn_benchmark_aug_train_results.csv', index = False)

json.dump(history_benchmark_aug.history, open(RESULTS_PATH + "history/benchmark_aug_cnn_history.json", "w"))


2024-09-01 12:29:07.753542: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1881600000 exceeds 10% of free system memory.


Epoch 1/12
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 45s 9ms/step - accuracy: 0.2008 - auc: 0.6659 - fn: 308275.6562 - fp: 161.5010 - loss: 2.2286 - precision: 0.9211 - recall: 0.0128 - tn: 2791278.0000 - tp: 1884.2921 - val_accuracy: 0.4910 - val_auc: 0.8069 - val_fn: 9971.0000 - val_fp: 2.0000 - val_loss: 1.8614 - val_precision: 0.9355 - val_recall: 0.0029 - val_tn: 89998.0000 - val_tp: 29.0000
Epoch 2/12
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 38s 8ms/step - accuracy: 0.4373 - auc: 0.8260 - fn: 277100.7188 - fp: 4788.4619 - loss: 1.7452 - precision: 0.8457 - recall: 0.0583 - tn: 2696651.0000 - tp: 23059.2285 - val_accuracy: 0.5380 - val_auc: 0.8596 - val_fn: 7900.0000 - val_fp: 221.0000 - val_loss: 1.5138 - val_precision: 0.9048 - val_recall: 0.2100 - val_tn: 89779.0000 - val_tp: 2100.0000
Epoch 3/12
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 39s 8ms/step - accuracy: 0.5202 - auc: 0.8729 - fn: 224401.6406 - fp: 19780.8477 - loss: 1.4707 - precision: 0.7935 - recall: 0.2430 - tn: 2681658.7500 - tp: 75758.3047 - val

### CNN

#### Normal

In [9]:
# callbacks cnn
callbacks_cnn = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_cnn = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_cnn = model_cnn.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_cnn
          )


# realiza predicao
train_pred_proba = model_cnn.predict(X_train)
test_pred_proba = model_cnn.predict(X_test)

# salva predicao no treino
test_results_cnn = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_cnn.to_csv(RESULTS_PATH + 'test/cnn_test_results.csv', index = False)

# salva predicao no teste
train_results_cnn = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_cnn.to_csv(RESULTS_PATH + 'train/cnn_train_results.csv', index = False)


json.dump(history_cnn.history, open(RESULTS_PATH + "history/cnn_history.json", "w"))

Epoch 1/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.1023 - auc: 0.7101 - fn: 34308.5898 - fp: 768.0000 - loss: 2.3037 - precision: 0.8837 - recall: 0.1884 - tn: 360524.3125 - tp: 5835.0000 - val_accuracy: 0.2417 - val_auc: 0.6698 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.2634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2130 - auc: 0.6576 - fn: 30143.5918 - fp: 0.0000e+00 - loss: 2.2409 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 271292.3125 - tp: 0.0000e+00 - val_accuracy: 0.3417 - val_auc: 0.7452 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.2095 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3087 - auc: 0.7426 - fn: 30143.5918 - fp: 0.0000e+00 - loss: 2.1661

#### Com data augmentation

In [10]:
# callbacks cnn
callbacks_cnn_aug = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_CNN_aug.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_CNN_aug.csv')
]

# limpa sessao do keras
keras.backend.clear_session()
gc.collect()

# cria modelo
model_cnn_aug = create_cnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_cnn_aug = model_cnn_aug.fit(X_train_augmented, y_train_augmented,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_cnn
          )


# realiza predicao
train_pred_proba = model_cnn_aug.predict(X_train)
test_pred_proba = model_cnn_aug.predict(X_test)

# salva predicao no treino
test_results_cnn_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_cnn_aug.to_csv(RESULTS_PATH + 'test/cnn_aug_test_results.csv', index = False)

# salva predicao no teste
train_results_cnn_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_cnn_aug.to_csv(RESULTS_PATH + 'train/cnn_aug_train_results.csv', index = False)


json.dump(history_cnn_aug.history, open(RESULTS_PATH + "history/cnn_aug_history.json", "w"))



Epoch 1/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 58s 11ms/step - accuracy: 0.2043 - auc: 0.6897 - fn: 301269.7812 - fp: 694.3376 - loss: 2.2128 - precision: 0.9278 - recall: 0.0590 - tn: 2790745.2500 - tp: 8890.1582 - val_accuracy: 0.4943 - val_auc: 0.8070 - val_fn: 9532.0000 - val_fp: 15.0000 - val_loss: 1.7931 - val_precision: 0.9689 - val_recall: 0.0468 - val_tn: 89985.0000 - val_tp: 468.0000 - learning_rate: 0.0010
Epoch 2/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 51s 11ms/step - accuracy: 0.4530 - auc: 0.8364 - fn: 260320.3594 - fp: 8201.6943 - loss: 1.6707 - precision: 0.8400 - recall: 0.1145 - tn: 2693237.7500 - tp: 39839.5859 - val_accuracy: 0.5350 - val_auc: 0.8567 - val_fn: 7799.0000 - val_fp: 260.0000 - val_loss: 1.5242 - val_precision: 0.8944 - val_recall: 0.2201 - val_tn: 89740.0000 - val_tp: 2201.0000 - learning_rate: 0.0010
Epoch 3/1000
4688/4688 ━━━━━━━━━━━━━━━━━━━━ 50s 11ms/step - accuracy: 0.5218 - auc: 0.8755 - fn: 219831.9688 - fp: 20717.7227 - loss: 1.4548 - precision: 0.7958

### Fully Connected

#### Normal

In [11]:
# callbacks cnn
callbacks_fcnn = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_fcnn.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_fcnn.csv')
]

# limpa sessao do keras
keras.backend.clear_session()

# cria modelo
model_fcnn = create_fcnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_fcnn = model_fcnn.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_fcnn
          )


# realiza predicao
train_pred_proba = model_fcnn.predict(X_train)
test_pred_proba = model_fcnn.predict(X_test)

# salva predicao no treino
test_results_fcnn = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_fcnn.to_csv(RESULTS_PATH + 'test/fcnn_test_results.csv', index = False)


# salva predicao no teste
train_results_fcnn = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_fcnn.to_csv(RESULTS_PATH + 'train/fcnn_train_results.csv', index = False)


json.dump(history_fcnn.history, open(RESULTS_PATH + "history/fcnn_history.json", "w"))

/home/semcovici/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/semcovici/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.1033 - auc: 0.6882 - fn: 34539.3320 - fp: 1723.2085 - loss: 2.5701 - precision: 0.7689 - recall: 0.1802 - tn: 359569.1250 - tp: 5604.2598 - val_accuracy: 0.1143 - val_auc: 0.5401 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.3004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1052 - auc: 0.5137 - fn: 30082.1934 - fp: 482.7851 - loss: 2.5004 - precision: 0.1092 - recall: 0.0021 - tn: 270809.5312 - tp: 61.3979 - val_accuracy: 0.1429 - val_auc: 0.5685 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.2784 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1139 - auc: 0.5231 - fn: 30109.7402 - fp: 294.7660 - loss: 2.4523 - precision: 

#### com data augmentation

In [12]:
# callbacks cnn
callbacks_fcnn_aug = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(PATH_MODELS + f'best_model_fcnn_aug.keras', save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    TensorBoard(log_dir=PATH_LOGS),
    CSVLogger(PATH_MODELS + f'training_log_fcnn_aug.csv')
]

# limpa sessao do keras
keras.backend.clear_session()

# cria modelo
model_fcnn_aug = create_fcnn_model(
    input_shape=input_shape,
    num_classes= num_classes,
    metrics = METRICS
)

# treina modelo
history_fcnn_aug = model_fcnn_aug.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_fcnn_aug
          )


# realiza predicao
train_pred_proba = model_fcnn_aug.predict(X_train)
test_pred_proba = model_fcnn_aug.predict(X_test)

# salva predicao no treino
test_results_fcnn_aug = pd.DataFrame({
    "y_test": y_test.argmax(axis=1),
    "y_test_pred": np.argmax(test_pred_proba, axis =1),
    "y_test_pred_proba": list(test_pred_proba)
    
})
test_results_fcnn_aug.to_csv(RESULTS_PATH + 'test/fcnn_aug_test_results.csv', index = False)


# salva predicao no teste
train_results_fcnn_aug = pd.DataFrame({
    "y_train": y_train.argmax(axis=1),
    "y_train_pred": np.argmax(train_pred_proba, axis =1),
    "y_train_pred_proba": list(train_pred_proba)
})
train_results_fcnn_aug.to_csv(RESULTS_PATH + 'train/fcnn_aug_train_results.csv', index = False)


json.dump(history_fcnn_aug.history, open(RESULTS_PATH + "history/fcnn_aug_history.json", "w"))

Epoch 1/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.0962 - auc: 0.7178 - fn: 32131.0449 - fp: 2908.6616 - loss: 2.7080 - precision: 0.7404 - recall: 0.2570 - tn: 358383.6562 - tp: 8012.5469 - val_accuracy: 0.0937 - val_auc: 0.5152 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.3313 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1021 - auc: 0.5092 - fn: 30033.2715 - fp: 981.1000 - loss: 2.5801 - precision: 0.1002 - recall: 0.0039 - tn: 270311.2188 - tp: 110.3191 - val_accuracy: 0.1042 - val_auc: 0.5386 - val_fn: 10000.0000 - val_fp: 0.0000e+00 - val_loss: 2.3009 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 90000.0000 - val_tp: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/1000
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.1011 - auc: 0.5142 - fn: 30083.9785 - fp: 567.7872 - loss: 2.5155 - precision: